# Web Scraping Medium's Tech Page


In [32]:
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
import pandas as pd
import requests
import re
import os
import urllib.request
import time

# Extract, clean, and store article HTML links

We take the first x articles.

In [20]:
#STORE ALL PAGES 

driver = webdriver.Chrome()
driver.get('https://medium.com/topic/technology')

#code based off of stackoverflow code #4 - need to add 
time.sleep(1)
elem = driver.find_element_by_tag_name("body")
pagedowns = 1000
while pagedowns:
    elem.send_keys(Keys.PAGE_DOWN)
    time.sleep(0.5)
    pagedowns-=1
    
pages = driver.page_source

In [21]:
len(pages)

9485434

In [22]:
#EXTRACT ARTICLE URLS FROM PAGES

soup = bs(pages, 'html5lib')
page_links = []


#collect all articles 
all_articles = soup.find_all('section')

for article in all_articles:
    hyperlink = article.contents[0].find_all('a')[0]
    
    #CLEAN UP URL
    #checks if href already contains full url
    if 'https://' in hyperlink.get('href'):
        url = hyperlink.get('href')
    else:
        url = 'https://medium.com' + hyperlink.get('href')
    
    #checks against duplicates
    if url not in page_links:
        page_links.append(url)


In [24]:
len(page_links)

3212

In [25]:
#STORE all cleaned htmls into csv
df = pd.DataFrame(page_links)
df.to_csv('cleaned_tech_htmls.csv', index=False)

# Scraping the Articles

In [ ]:
#Jesse's parser code

In [61]:
#turn csv of htmls into a list and make dictionary to hold data
tech_htmls = pd.read_csv("cleaned_tech_htmls.csv")
tech_html_list = tech_htmls['0'].tolist()
htmls_with_data = {}

In [63]:
#making for loop to scrape the data:
counter = 1

for i in tech_html_list:
    time.sleep(5)
    #page request
    page_response = requests.get(i, timeout=5)
    print(page_response)
    
    #parsing content from page and storing it
    page_content = bs(page_response.content, "html.parser")
    
    #empty list to store user data
    data = []
    
    #try to grab header
    try:
        #grabs div that stores title info
        titleInfo = page_content.find_all('div',attrs={"class":"n p"})

        #finds title within titleInfo div
        href = str(titleInfo).index("<h1")
        start = str(titleInfo)[href:].index(">")+href+1
        end = str(titleInfo)[href:].index("</h1>")+href

        #appends title to data list after removing extra tags
        title = re.sub(re.compile('<.*?>'), '', str(titleInfo)[start:end])
        data.append(title)
    except:
        data.append(None)
    #try to grab author
    try:

        #<h1>Finding and Adding Author to Data</h1>

        #grabs div that stores user info
        userInfo = page_content.find_all('div',attrs={"class":"o n"})

        #finds authors name within userInfo div
        href = str(userInfo)[24:].index("<a class=")+24
        start = str(userInfo)[href:].index(">")+href+1
        end = str(userInfo)[href:].index("</a>")+href

        #appends authors name to data list
        author = str(userInfo)[start:end]
        data.append(author)
    except:
        data.append(None)
    try:
        #trying to get author page url:

        #<h1>Finding and Adding Author Page URL to Data</h1>

        #finds date within userInfo div
        href = str(userInfo)[24:].index("<a class=")
        start = str(userInfo)[href:].index("href=")+href+6
        end = str(userInfo)[start:].index("?source")+start

        #appends date to data list
        url = "medium.com" + str(userInfo)[start:end]
        data.append(url)
    except:
        data.append(None)
        
    ### PARSING THROUGH USE PAGE NOW ###
    try: #adding user data:
        #user url
        user_link = "https://" + data[2]
        #user page request
        time.sleep(5)
        user_response = requests.get(user_link, timeout=5)
        #parsing content from user page to store follower count
        user_content = BeautifulSoup(user_response.content, "html.parser")
        #user tag
        user_href = re.sub("medium.com", "", data[2])
    except:
        pass
    
    try: #get member since
        memberInfo = user_content.find_all("span")
        year = re.findall("[0-9]{4}", str(memberInfo))[0]
        data.append(year)
    except:
        data.append(None)
    
    try: #get following:
        #returns following count and adds to data list
        followingInfo = user_content.find_all('a', attrs={"href":user_href+"/following"})
        following = re.sub("[^0-9]", "", str(followingInfo))
        data.append(following)
    except:
        data.append(None)
        
    #get follower count
    try: 
        #returns follower count and adds to data list
        followerInfo = user_content.find_all('a', attrs={"href":user_href+"/followers"})
        followers = re.sub("[^0-9]", "", str(followerInfo))
        data.append(followers)
    except:
        data.append(None)
        
    #trying to add date
    try:
        #<h1>Finding and Adding Date to Data</h1>

        #finds date within userInfo div
        href = str(userInfo)[end:].index("<a class=")+end
        start = str(userInfo)[href:].index(">")+href+1
        end = str(userInfo)[href:].index("</a>")+href

        #appends date to data list
        date = str(userInfo)[start:end]
        data.append(date)
    except:
        data.append(None)

    #tring to add clap number
    try:
        #<h1>Finding and Adding Number of Claps to Data</h1>

        #grabs all paragraphs in the article
        clapInfo = page_content.find_all('div',attrs={"class":"n o"})

        #finds claps within clapInfo div
        result = re.sub(re.compile('<.*?>'), '', str(clapInfo))
        result = re.findall("\d+", result)

        #appends claps to data list
        if len(result) == 0:
            data.append('0')

        else:
            data.append(result[0])
    except:
        data.append(None)
        
    #getting all the text data:
    try:
        
        #<h1>Adding a List of the Text Content to Data</h1>

        #grabs all paragraphs in the article
        textContent = page_content.find_all("p")
        textInfo = str(textContent)

        #RegEx that cleans paragraph data by removing html tags and extra commas
        result = re.sub(re.compile('<.*?>'), '', textInfo)
        result = re.split("\., ", result)
        result = result[:-1]
        data.append(result)
    except:
        data.append(None)
    
    #append html to dictionary with data
    htmls_with_data[i] = data

    
    #counter
    print(counter)
    counter = counter + 1

<Response [200]>
1
<Response [200]>
2
<Response [200]>
3
<Response [200]>
4
<Response [200]>
5
<Response [200]>
6
<Response [200]>
7
<Response [200]>
8
<Response [200]>
9
<Response [200]>
10
<Response [200]>
11
<Response [200]>
12
<Response [200]>
13
<Response [200]>
14
<Response [200]>
15
<Response [200]>
16
<Response [200]>
17
<Response [200]>
18
<Response [200]>
19
<Response [200]>
20
<Response [200]>
21
<Response [200]>
22
<Response [200]>
23
<Response [200]>
24
<Response [200]>
25
<Response [200]>
26
<Response [200]>
27
<Response [200]>
28
<Response [200]>
29
<Response [200]>
30
<Response [200]>
31
<Response [200]>
32
<Response [200]>
33
<Response [200]>
34
<Response [200]>
35
<Response [200]>
36
<Response [200]>
37
<Response [200]>
38
<Response [200]>
39
<Response [200]>
40
<Response [200]>
41
<Response [200]>
42
<Response [200]>
43
<Response [200]>
44
<Response [200]>
45
<Response [200]>
46
<Response [200]>
47
<Response [200]>
48
<Response [200]>
49
<Response [200]>
50
<Response

396
<Response [200]>
397
<Response [200]>
398
<Response [200]>
399
<Response [200]>
400
<Response [200]>
401
<Response [200]>
402
<Response [200]>
403
<Response [200]>
404
<Response [200]>
405
<Response [200]>
406
<Response [200]>
407
<Response [200]>
408
<Response [200]>
409
<Response [200]>
410
<Response [200]>
411
<Response [200]>
412
<Response [200]>
413
<Response [200]>
414
<Response [200]>
415
<Response [200]>
416
<Response [200]>
417
<Response [200]>
418
<Response [200]>
419
<Response [200]>
420
<Response [200]>
421
<Response [200]>
422
<Response [200]>
423
<Response [200]>
424
<Response [200]>
425
<Response [200]>
426
<Response [200]>
427
<Response [200]>
428
<Response [200]>
429
<Response [200]>
430
<Response [200]>
431
<Response [200]>
432
<Response [200]>
433
<Response [200]>
434
<Response [200]>
435
<Response [200]>
436
<Response [200]>
437
<Response [200]>
438
<Response [200]>
439
<Response [200]>
440
<Response [200]>
441
<Response [200]>
442
<Response [200]>
443
<Response

<Response [200]>
787
<Response [200]>
788
<Response [200]>
789
<Response [200]>
790
<Response [200]>
791
<Response [200]>
792
<Response [200]>
793
<Response [200]>
794
<Response [200]>
795
<Response [200]>
796
<Response [200]>
797
<Response [200]>
798
<Response [200]>
799
<Response [200]>
800
<Response [200]>
801
<Response [200]>
802
<Response [200]>
803
<Response [200]>
804
<Response [200]>
805
<Response [200]>
806
<Response [200]>
807
<Response [200]>
808
<Response [200]>
809
<Response [200]>
810
<Response [200]>
811
<Response [200]>
812
<Response [200]>
813
<Response [200]>
814
<Response [200]>
815
<Response [200]>
816
<Response [200]>
817
<Response [200]>
818
<Response [200]>
819
<Response [200]>
820
<Response [200]>
821
<Response [200]>
822
<Response [200]>
823
<Response [200]>
824
<Response [200]>
825
<Response [200]>
826
<Response [200]>
827
<Response [200]>
828
<Response [200]>
829
<Response [200]>
830
<Response [200]>
831
<Response [410]>
832
<Response [200]>
833
<Response [20

1169
<Response [200]>
1170
<Response [200]>
1171
<Response [200]>
1172
<Response [200]>
1173
<Response [200]>
1174
<Response [200]>
1175
<Response [200]>
1176
<Response [200]>
1177
<Response [200]>
1178
<Response [200]>
1179
<Response [200]>
1180
<Response [200]>
1181
<Response [200]>
1182
<Response [200]>
1183
<Response [200]>
1184
<Response [200]>
1185
<Response [200]>
1186
<Response [200]>
1187
<Response [200]>
1188
<Response [200]>
1189
<Response [200]>
1190
<Response [200]>
1191
<Response [200]>
1192
<Response [200]>
1193
<Response [200]>
1194
<Response [200]>
1195
<Response [200]>
1196
<Response [200]>
1197
<Response [200]>
1198
<Response [200]>
1199
<Response [200]>
1200
<Response [200]>
1201
<Response [200]>
1202
<Response [200]>
1203
<Response [200]>
1204
<Response [200]>
1205
<Response [200]>
1206
<Response [200]>
1207
<Response [200]>
1208
<Response [200]>
1209
<Response [200]>
1210
<Response [200]>
1211
<Response [200]>
1212
<Response [200]>
1213
<Response [200]>
1214
<Resp

1542
<Response [200]>
1543
<Response [200]>
1544
<Response [200]>
1545
<Response [200]>
1546
<Response [200]>
1547
<Response [200]>
1548
<Response [200]>
1549
<Response [200]>
1550
<Response [200]>
1551
<Response [200]>
1552
<Response [200]>
1553
<Response [200]>
1554
<Response [200]>
1555
<Response [200]>
1556
<Response [200]>
1557
<Response [200]>
1558
<Response [200]>
1559
<Response [200]>
1560
<Response [200]>
1561
<Response [200]>
1562
<Response [200]>
1563
<Response [200]>
1564
<Response [200]>
1565
<Response [200]>
1566
<Response [200]>
1567
<Response [200]>
1568
<Response [200]>
1569
<Response [200]>
1570
<Response [200]>
1571
<Response [200]>
1572
<Response [200]>
1573
<Response [200]>
1574
<Response [200]>
1575
<Response [200]>
1576
<Response [200]>
1577
<Response [200]>
1578
<Response [200]>
1579
<Response [200]>
1580
<Response [200]>
1581
<Response [200]>
1582
<Response [200]>
1583
<Response [200]>
1584
<Response [200]>
1585
<Response [200]>
1586
<Response [200]>
1587
<Resp

ReadTimeout: HTTPSConnectionPool(host='medium.com', port=443): Read timed out. (read timeout=5)